<a href="https://colab.research.google.com/github/JonatanPolanco/Data_Quality_Testing/blob/main/Data_Quality_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install --target=$nb_path pydeequ==1.0.1         ##Solo correr una vez

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
!pip install --target=$nb_path pyspark==3.0.3         ##Solo correr una vez

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
pip install qvd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from pyspark.sql import SparkSession, Row
import pydeequ
import pandas as pd
import re
from qvd import qvd_reader
from pyspark.sql.types import IntegerType
from pydeequ.checks import *
from pydeequ.verification import *
import numpy as np

In [35]:
spark = (SparkSession
    .builder
    .appName("Example") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.memory", "16g")\
    .config("spark.executor.memory", "4g") \
    .getOrCreate())
check = Check(spark, CheckLevel.Warning, "Review Check")

In [36]:
df_param = pd.read_excel("Parametros.xlsx",sheet_name="Validaciones")
df_Person_Person = pd.read_csv('dbo_Person_Person.csv')
df_address = pd.read_csv('dbo_Address.csv')

In [37]:
df_param_ = df_param.loc[df_param['Tipo Validacion'] == 'Formato']
tablas = df_param_.drop_duplicates(subset=['Esquema', 'Tabla'])
tablas = np.array(tablas["Esquema"]+"_"+tablas["Tabla"])
tablas_campo = df_param_["Esquema"]+"_"+df_param_["Tabla"]+"."+df_param_["Campo"]
df_param_["Valores"]=df_param_["Valores"].replace("TipoDato", "hasDataType")
df_param_["Valores"]=df_param_["Valores"].replace("MinLength", "hasMinLength")
df_param_["Valores"]=df_param_["Valores"].replace("MaxLength", "hasMaxLength")
df_param_["Valores"]=df_param_["Valores"].replace("Email", "containsEmail")
df_param_["Valores"]=df_param_["Valores"].replace("MaxNumRegistros", "hasSize")
df_param_["Valores"]=df_param_["Valores"].replace("CreditCardNumber", "containsCreditCardNumber")
df_param_["Valores"]=df_param_["Valores"].replace("URL", "containsURL")
df_param_["Valores"]=df_param_["Valores"].replace("SocialSecurityNumber", "containsSocialSecurityNumber")

<ipython-input-37-836ec9f8d57e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_param_["Valores"]=df_param_["Valores"].replace("TipoDato", "hasDataType")
<ipython-input-37-836ec9f8d57e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_param_["Valores"]=df_param_["Valores"].replace("MinLength", "hasMinLength")
<ipython-input-37-836ec9f8d57e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [38]:
df_param_

,Esquema,Tabla,Campo,Tipo Validacion,Valores,Formato,Prioridad,Dimension,Unnamed: 8
3,dbo,Person_Person,FirstName,Formato,hasDataType,STRING,NaN,NaN,NaN
4,dbo,Person_Person,Email,Formato,containsEmail,NaN,NaN,NaN,NaN
5,dbo,Person_Person,Password,Formato,hasMinLength,10,NaN,NaN,NaN
6,dbo,Person_Person,Password,Formato,hasMaxLength,100,NaN,NaN,NaN
7,dbo,Address,Date,Formato,dd/mm/aaaa,NaN,NaN,NaN,NaN
8,dbo,Address,LlaveCompuestaD,Formato,hasMaxLength,15,NaN,NaN,NaN


In [39]:
data = {}
for i in range(len(tablas)):
  data[i] = pd.read_csv(tablas[i]+'.csv')

In [40]:
result_df = {}
result_df_ = pd.DataFrame()
for i in range(len(df_param_)):
  for j in range(len(tablas)):

    if df_param_.iloc[i]["Valores"] == 'hasMaxLength':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        print(df_param_.iloc[i]["Formato"])
        check_ = Check(spark, CheckLevel.Warning, "hasMaxLength").hasMaxLength(df_param_.iloc[i]["Campo"],lambda x: x == df_param_.iloc[i]["Formato"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    elif df_param_.iloc[i]["Valores"] == 'hasMinLength':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "hasMinLength").hasMinLength(df_param_.iloc[i]["Campo"],lambda x: x == df_param_.iloc[i]["Formato"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    elif df_param_.iloc[i]["Valores"] == 'containsEmail':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "containsEmail").containsEmail(df_param_.iloc[i]["Campo"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    elif df_param_.iloc[i]["Valores"] == 'containsURL':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "containsURL").containsURL(df_param_.iloc[i]["Campo"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    elif df_param_.iloc[i]["Valores"] == 'hasSize':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "hasSize").hasSize(df_param_.iloc[i]["Campo"], lambda x: x >= df_param_.iloc[i]["Formato"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    #elif df_param_.iloc[i]["Valores"] == 'hasDataType':
     # if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
      #  check_ = Check(spark, CheckLevel.Warning, "hasDataType").hasDataType(df_param_.iloc[i]["Campo"], df_param_.iloc[i]["Formato"])
       # result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        #result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_)

    elif df_param_.iloc[i]["Valores"] == 'containsCreditCardNumber':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "containsCreditCardNumber").containsCreditCardNumber(df_param_.iloc[i]["Campo"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

    elif df_param_.iloc[i]["Valores"] == 'containsSocialSecurityNumber':
      if df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"] == tablas[j]:
        check_ = Check(spark, CheckLevel.Warning, "containsSocialSecurityNumber").containsSocialSecurityNumber(df_param_.iloc[i]["Campo"])
        result_ = VerificationSuite(spark).onData(data[j]).addCheck(check_).run()
        result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

/usr/local/lib/python3.8/dist-packages/pydeequ/pandas_utils.py:26: UserWarning: WARNING: You passed in a Pandas DF, so we will be using our experimental utility to convert it to a PySpark DF.
  warnings.warn(


100
15


In [41]:
result_df_= pd.concat(result_df)
result_df_

,,check,check_level,check_status,constraint,constraint_status,constraint_message
1,0,containsEmail,Warning,Success,containsEmail(Email),Success,
2,0,hasMinLength,Warning,Warning,"MinLengthConstraint(MinLength(Password,None))",Failure,Value: 8.0 does not meet the constraint requir...
3,0,hasMaxLength,Warning,Warning,"MaxLengthConstraint(MaxLength(Password,None))",Failure,Value: 15.0 does not meet the constraint requi...
5,0,hasMaxLength,Warning,Success,"MaxLengthConstraint(MaxLength(LlaveCompuestaD,...",Success,


In [42]:
df_address.head(10000)
print("Longest length is:\n",df_address.LlaveCompuestaD.str.len().max())

Longest length is:
 15


In [43]:
df_Person_Person.head()
print("shortes length is:\n",df_Person_Person.Password.str.len().min())

shortes length is:
 8


In [44]:
data[0].Email[0]

'gwyn.etzel@aol.com'